In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pinocchio as pin
import meshcat
import re
from pinocchio.robot_wrapper import RobotWrapper

from utils import *
from inverse_kinematics import *

## Initialize the Robot and the Visualizer

In [3]:
%%capture
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess()
viewer = meshcat.Visualizer(zmq_url=zmq_url)

## comment out the following code if you're running the notebook locally
ip_addr_regex = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
public_ip = '137.184.48.127'
public_url = re.sub(ip_addr_regex, public_ip, web_url)
viewer.window.web_url = public_url

In [4]:
robot = RobotWrapper.BuildFromURDF('./urdf/arm.urdf', './urdf')
viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model
)
viz.initViewer(viewer)
viz.loadViewerModel()
print('You can open the visualizer by visiting the following URL:')
print(viewer.window.web_url)

You can open the visualizer by visiting the following URL:
http://137.184.48.127:7000/static/


## Load Data

In [5]:
config = {'vertical': ['Top', 'Middle','Bottom'],
          'horizontal': ['Left', 'Center', 'Right'],
          'traj_ids': [1, 2],
          'subject_ids': [2, 3, 4]}

In [6]:
traj = getExtendedTraj(0, 1, "PositionTopCenterTrajectory1Subject3", config)

## Inverse Kinematics for a Single Frame (Wrist)

In [7]:
def calculateJointTrajWrist(traj):
    wrist_pos = traj[:, 17:20]
    shoulder_pos = traj[:, 11:14]
    wrist_rel = 1 / 1000 * (wrist_pos - shoulder_pos)
    q = np.zeros(5)
    traj_q = np.zeros((len(wrist_rel), 5))
    frame_id = robot.model.getFrameId('wrist')
    q_previous = findAngle(wrist_rel[0, :], q, robot, frame_id)          

    for n in range(len(wrist_rel)):
        p_des = wrist_rel[n, :]
        q = findAngle(p_des, q, robot, frame_id)
        
        if isinstance(q, bool):
            q = q_previous
        else: 
            q_previous = findAngle(p_des, q, robot, frame_id)          
            
        traj_q[n, :] = q_previous
    
    return traj_q

In [8]:
traj_q = calculateJointTrajWrist(traj)

In [9]:
visualizeJointTraj(traj_q, viz)

## Inverse Kinematics for Two Frames (Wrist and Elbow)

In [10]:
def calculateJointTrajWristElbow(traj):
    wrist_pos = traj[:, 17:20]
    elbow_pos = traj[:, 5:8]
    shoulder_pos = traj[:, 11:14]
    wrist_rel = 1 / 1000 * (wrist_pos - shoulder_pos)
    elbow_rel = 1 / 1000 * (elbow_pos - shoulder_pos)
    
    traj_q = np.zeros((len(wrist_rel), 5))
    frame_id_wrist = robot.model.getFrameId('wrist')
    frame_id_elbow = robot.model.getFrameId('elbow')
    angles_previous = findAngleTwoFrames(robot, wrist_rel[0, :], elbow_rel[0, :], 
                                         frame_id_wrist, frame_id_elbow, np.zeros(5))
    
    for n in range(len(wrist_rel)):
        p_des_wrist = wrist_rel[n, :]
        p_des_elbow = elbow_rel[n, :]
        angles = findAngleTwoFrames(robot, p_des_wrist, p_des_elbow, 
                                    frame_id_wrist, frame_id_elbow, angles_previous)
        
        if isinstance(angles, bool):
            angles = angles_previous
        else: 
            angles_previous = findAngleTwoFrames(robot, p_des_wrist, p_des_elbow, 
                                                 frame_id_wrist, frame_id_elbow, angles_previous)          
            
        traj_q[n, :] = angles


    return traj_q

In [11]:
traj_q = calculateJointTrajWristElbow(traj)
visualizeJointTraj(traj_q, viz)